## Functions

#### Overview

In [1]:
def add_one(number):
    return number + 1


add_one(2)

3

#### Functions are First-Class objects

In [2]:
# In Python, functions are first-class objects. This means that functions can be passed around and used as arguments


def say_hello(name):
    return f"Hello {name}"


def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"


def greet_bob(greeter_func):
    return greeter_func("Bob")

In [3]:
greet_bob(say_hello)

'Hello Bob'

In [4]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

#### Inner Functions

In [5]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()


parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


The inner functions are not defined until the parent function is called.
They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. 

#### Returning Functions From Functions

In [6]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child


first = parent(1)
second = parent(2)

In [7]:
first

<function __main__.parent.<locals>.first_child()>

In [8]:
second

<function __main__.parent.<locals>.second_child()>

In [9]:
# You can now use first and second as if they are regular functions, even though the functions they point to can’t be accessed directly.
print(first())
print(second())

Hi, I am Emma
Call me Liam


## Simple Decorators

In [10]:
def my_decorator(func):  # Pass in the function to be decorated.
    def wrapper():  # Define the function *that we will return*.
        print("Something is happening before the function is called.")
        func()  # Call the decorated function inside.
        print("Something is happening after the function is called.")

    return wrapper  # Return the decorated function.


def say_whee():
    print("Whee!")


say_whee = my_decorator(say_whee)  # "Manually" decorate the function.

In [11]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [12]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

Put simply:

**Decorators wrap a function, modifying its behavior.**

In [13]:
# Another example.

from datetime import datetime


def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep

    return wrapper


def say_whee():
    print("Whee!")


say_whee = not_during_the_night(say_whee)

say_whee()

Whee!


#### Syntactic sugar: `@` ("pie" syntax)

In [14]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")

    return wrapper


@my_decorator
def say_whee():
    print("Whee!")

#### Reusing Decorators

In [15]:
# See decorators.py
from decorators import do_twice

In [16]:
@do_twice
def say_whee():
    print("Whee!")

In [17]:
say_whee()

Whee!
Whee!


#### Decorating Functions With Arguments

In [20]:
from decorators import do_twice


@do_twice
def greet(name):
    print(f"Hello {name}")

In [19]:
greet("World")

# This won't work as `do_twice` expects `func` to have no arguments.

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

In [27]:
# Use *args, **kwargs to handle this.


def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)

    return wrapper_do_twice

In [22]:
@do_twice
def say_whee():
    print("Whee!")


@do_twice
def greet(name):
    print(f"Hello {name}")

In [24]:
say_whee()

Whee!
Whee!


In [25]:
greet("World")

Hello World
Hello World


#### Returning Values From Decorated Functions

In [28]:
# In this case, the decorator will swallow the return of the decorated function.


@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"


hi_adam = return_greeting("Adam")
print(hi_adam)  # Oops! Get None.

Creating greeting
Creating greeting
None


In [30]:
# To fix, return as follows.


def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)  # Here.

    return wrapper_do_twice


@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"


return_greeting("Adam")  # Now get the returned value as expected.

Creating greeting
Creating greeting


'Hi Adam'

#### On Introspection

In [31]:
print

<function print>

In [32]:
print.__name__

'print'

In [33]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



⚠️ Decorators, if implemented naively, will break such function introspection as above.

To fix: use use `@functools.wraps`:

In [36]:
import functools


def do_twice(func):
    @functools.wraps(func)  # NOTE: functools wraps is applied >>> on the wrapper <<< (not on the decorator).
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)

    return wrapper_do_twice


@do_twice
def say_whee():
    print("Whee!")

In [37]:
say_whee

<function __main__.say_whee()>

In [38]:
say_whee.__name__

'say_whee'

In [39]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



## A Few Real World Examples

### Timing functions

In [1]:
# Useful decorator boilerplate:

import functools


def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value

    return wrapper_decorator

In [2]:
import functools
import time


def timer(func):
    """Print the runtime of the decorated function"""

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer


@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [3]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0017 secs


In [4]:
waste_some_time(999)

Finished 'waste_some_time' in 1.4298 secs


#### Debugging Code

In [5]:
import functools


def debug(func):
    """Print the function signature and return value"""

    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]  # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)  # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")  # 4
        return value

    return wrapper_debug

In [6]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [7]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [8]:
make_greeting("Richard", age=112)

Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'


'Whoa Richard! 112 already, you are growing up!'

In [9]:
make_greeting(name="Dorrisile", age=116)

Calling make_greeting(name='Dorrisile', age=116)
'make_greeting' returned 'Whoa Dorrisile! 116 already, you are growing up!'


'Whoa Dorrisile! 116 already, you are growing up!'

In [10]:
# Example where this might actually be useful...

import math
from decorators import debug

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)


def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [11]:
approximate_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

#### Slowing Down Code

In [12]:
# Most common use case is that you want to rate-limit a function that continuously checks whether
# a resource - like a web page - has changed.

import functools
import time


def slow_down(func):
    """Sleep 1 second before calling the function"""

    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)

    return wrapper_slow_down


@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [13]:
countdown(3)

3
2
1
Liftoff!


#### Registering Plugins

Decorators **don’t have to wrap the function they're decorating**. 

They can also *simply register that a function exists and return it unwrapped*.

This can be used, for instance, to create a light-weight plug-in architecture:

In [15]:
import random

PLUGINS = dict()


def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func  # Register `func`'s existence in the PLUGINS dict.
    return func  # `func` is returned unwrapped.


# NOTE: You do not have to write an inner function or use @functools.wraps in this example because you are
# returning the original function unmodified.


@register
def say_hello(name):
    return f"Hello {name}"


@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"


def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [16]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [17]:
randomly_greet("Alice")

Using 'say_hello'


'Hello Alice'

In [21]:
# PS, a bit similar to `globals()`
g = globals()
print(str(g)[:1000], "\n...")

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', '# Useful decorator boilerplate:\n\nimport functools\n\ndef decorator(func):\n    @functools.wraps(func)\n    def wrapper_decorator(*args, **kwargs):\n        # Do something before\n        value = func(*args, **kwargs)\n        # Do something after\n        return value\n    return wrapper_decorator', 'import functools\nimport time\n\ndef timer(func):\n    """Print the runtime of the decorated function"""\n    @functools.wraps(func)\n    def wrapper_timer(*args, **kwargs):\n        start_time = time.perf_counter()    # 1\n        value = func(*args, **kwargs)\n        end_time = time.perf_counter()      # 2\n        run_time = end_time - start_time    # 3\n        print(f"Finished {func.__name__!r} in {run_time:.4f} sec

#### Is the User Logged In?

In this example, we are using `Flask` to set up a `/secret` web page that should only be visible to users that are logged in or otherwise authenticated.

In [2]:
from flask import Flask, g, request, redirect, url_for
import functools

app = Flask(__name__)


def login_required(func):
    """Make sure user is logged in before proceeding"""

    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)

    return wrapper_login_required


@app.route("/secret")
@login_required
def secret():
    return "Secret!"


# NOTE:
# This code is just an illustrative example, not really functional.

## Fancy Decorators

Covers:
* Decorators on classes
* Several decorators on one function
* Decorators with arguments
* Decorators that can optionally take arguments
* Stateful decorators
* Classes as decorators

#### Decorating Classes

---

##### Sidetrack - built-in decorators
Notable built-in decorators: `@classmethod`, `@staticmethod`, and `@property`.

In [3]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    # Property decorator - getter.
    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    # Property decorator - setter.
    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    # Example of an immutable property, no setter.
    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    # Class method, used as factory method.
    # Why class method here? I guess to call initializer cls(...), although could have used a static method.
    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    # Static method.
    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [5]:
c = Circle(5)
c.radius

5

In [6]:
c.area

78.5398163375

In [7]:
c.radius = 2
c.area

12.566370614

In [8]:
c.area = 100

AttributeError: can't set attribute

In [9]:
c.cylinder_volume(height=4)

50.265482456

In [10]:
c.radius = -1

ValueError: Radius must be positive

In [11]:
c = Circle.unit_circle()
c.radius

1

In [12]:
c.pi()

3.1415926535

In [13]:
Circle.pi()

3.1415926535

---

##### Decorating methods in class

In [14]:
from decorators import debug, timer


class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [15]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x7f34812d48b0>, 1000)
'__init__' returned None


In [16]:
tw.waste_time(999)

Finished 'waste_time' in 0.1377 secs


##### Decorating the whole class

In [17]:
from dataclasses import dataclass


@dataclass
class PlayingCard:
    rank: str
    suit: str

The meaning of the syntax is similar to the function decorators. 

In the example above, you could have done the decoration by writing `PlayingCard = dataclass(PlayingCard)`.

> A common use of class decorators is to be a **simpler alternative to some use-cases of metaclasses**.
>
> In both cases, you are changing the definition of a class dynamically.

Decorating a class does not decorate its methods.

In [23]:
# Writing your own class decorator (very much analogous to function decorators).

# Stupid example.

import time


def timer(cls):
    @functools.wraps(cls)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = cls(*args, **kwargs)  # Initializer of wrapped class triggered.
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {cls.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer


@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [24]:
tw = TimeWaster(1000)  # Decorator will trigger here...

Finished 'TimeWaster' in 0.0000 secs


In [25]:
tw.waste_time(999)  # ... And not here.

A better example (singleton) is to follow later in this tutorial.

#### Nesting Decorators

In [1]:
from decorators import debug, do_twice


@debug
@do_twice
def greet(name):
    print(f"Hello {name}")


# Decorators are executed in the order they are listed.
# In other words, @debug calls @do_twice, which calls greet(), or debug(do_twice(greet()))

In [2]:
greet("Eva")

Calling greet('Eva')
Hello Eva
Hello Eva
'greet' returned None


In [3]:
# Now if we change the order...

from decorators import debug, do_twice


@do_twice
@debug
def greet(name):
    print(f"Hello {name}")


greet("Eva")

Calling greet('Eva')
Hello Eva
'greet' returned None
Calling greet('Eva')
Hello Eva
'greet' returned None


#### Decorators With Arguments

⚠️ Can be tricky

In [4]:
# Naive approach.


def repeat_naive(func, num_times):  # Hmm, what do I do??
    ...


# Solution: either use `functools.partial()`, as per:
# https://github.com/dabeaz/python-cookbook/blob/master/src/9/defining_a_decorator_that_takes_an_optional_argument/example.py
# Or shove the decorator inside another function that handles the arguments, as shown below.

In [7]:
import functools


def repeat(num_times):  # Argument handler machine. Notice it *doesn't* take in `func`!
    def decorator_repeat(func):  # Decorator. Takes in `func`.
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):  # Using `num_times` argument here.
                value = func(*args, **kwargs)
            return value  # type: ignore

        return wrapper_repeat

    return decorator_repeat  # Argument handler machine returns the actual decorator.

In [8]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")


greet("Bob Lazar")

Hello Bob Lazar
Hello Bob Lazar
Hello Bob Lazar
Hello Bob Lazar


In [9]:
# And an example that can handle both cases:
#   * Decorator with argument(s),
#   * Decorator with no arguments.


def repeat(_func=None, *, num_times=2):  # Make use of keyword only * syntax.
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value  # type: ignore

        return wrapper_repeat

    # Here, decide:
    # NOTE: This is very unintuitive!
    if _func is None:
        # Case: `repeat` being used as decorator WITH arguments. Return the decorator.
        return decorator_repeat  # Case 1.
    else:
        # Case: `repeat` being used as decorator WITHOUT arguments. Return the decorated function.
        return decorator_repeat(_func)  # Case 2.

In [11]:
@repeat
def say_whee():  # Case 2 above.
    print("Whee!")


@repeat(num_times=3)  # Case 1 above.
def greet(name):
    print(f"Hello {name}")

In [12]:
say_whee()

Whee!
Whee!


In [13]:
greet("Penny")

Hello Penny
Hello Penny
Hello Penny


#### Stateful Decorators

In [14]:
# In the next section, you will see how to use classes to keep state.
# But in simple cases, you can also get away with using *function attributes*.
# In this case, we utilize wrapper_count_calls.num_calls.

import functools


def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)

    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls


@count_calls
def say_whee():
    print("Whee!")

In [15]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [16]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [17]:
say_whee.num_calls

2

### Classes *as* Decorators

The typical way to **maintain state** is by using classes.

In this section, you’ll see how to rewrite the `@count_calls` example from the previous section using a class as a decorator.

In [18]:
import functools


class CountCalls:
    def __init__(self, func):  # `func` to decorate gets passed in in the initializer.

        functools.update_wrapper(self, func)
        # NOTE: ^ This is a key difference. Use this approach to maintain function info.
        # Details: https://docs.python.org/library/functools.html#functools.update_wrapper

        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):  # The `__call__` method acts as `wrapper` did in function decorators.
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)


@CountCalls
def say_whee():
    print("Whee!")

In [19]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [20]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [21]:
say_whee.num_calls

2

In [26]:
# Note:
say_whee.__name__  # type: ignore

'say_whee'

In [29]:
# But note:
type(say_whee)

__main__.CountCalls

## More Real World Examples

#### Slowing Down Code, Revisited

In [30]:
# Now using a parameter: `rate`.

import functools
import time


def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""

    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)

        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

In [31]:
@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [32]:
countdown(3)

3
2
1
Liftoff!


#### Creating Singletons

> A singleton is a class with only one instance.

**Fun fact:**
* There are several singletons in Python that you use frequently, including `None`, `True`, and `False`.
* Using `is` returns `True` only for objects that are **the exact same instance**.
It is the fact that `None` is a singleton that allows you to compare for `None` using the `is` keyword!

<br/>

The following `@singleton` decorator turns a class into a singleton by storing the first instance of the class as 
an attribute.

Later attempts at creating an instance simply return the stored instance:

In [33]:
# Singleton implementation.

import functools


def singleton(cls):
    """Make a class a Singleton class (only one instance)"""

    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:  # (1) Checks whether .instance has been defined. If not, initialize class.
            wrapper_singleton.instance = cls(*args, **kwargs)  # Assign class to instance.
        return wrapper_singleton.instance  # Otherwise, return that initial instance.

    wrapper_singleton.instance = None  # This line is key, so as to not raise error at (1).

    return wrapper_singleton


@singleton
class TheOne:
    pass

In [34]:
first_one = TheOne()
another_one = TheOne()

In [35]:
id(first_one)

139808204558832

In [36]:
id(another_one)

139808204558832

In [38]:
first_one is another_one

# Works as a singleton, as expected.

True

> **Note:**
>
> Singleton classes are not really used as often in Python as in other languages. 
>
> The effect of a singleton is usually better implemented as a global variable in a module.

#### Caching Return Values

Decorators can provide a nice mechanism for **caching** and **memoization**.

As an example, let’s look at a recursive definition of the Fibonacci sequence.

In [1]:
from decorators import count_calls


@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [4]:
# Very bad performance due to repeated recursion of same argument calls.

fibonacci(6)

Call 16 of 'fibonacci'
Call 17 of 'fibonacci'
Call 18 of 'fibonacci'
Call 19 of 'fibonacci'
Call 20 of 'fibonacci'
Call 21 of 'fibonacci'
Call 22 of 'fibonacci'
Call 23 of 'fibonacci'
Call 24 of 'fibonacci'
Call 25 of 'fibonacci'
Call 26 of 'fibonacci'
Call 27 of 'fibonacci'
Call 28 of 'fibonacci'
Call 29 of 'fibonacci'
Call 30 of 'fibonacci'
Call 31 of 'fibonacci'
Call 32 of 'fibonacci'
Call 33 of 'fibonacci'
Call 34 of 'fibonacci'
Call 35 of 'fibonacci'
Call 36 of 'fibonacci'
Call 37 of 'fibonacci'
Call 38 of 'fibonacci'
Call 39 of 'fibonacci'
Call 40 of 'fibonacci'


8

In [5]:
fibonacci.num_calls

40

In [9]:
# Solution using caching.

import functools
from decorators import count_calls


def cache(func):
    """Keep a cache of previous function calls"""

    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())  # NOTE: Cache key is tuple of all args and kwargs.
        if cache_key not in wrapper_cache.cache:  # Use wrapper_cache.cache dict to store the cache.
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]

    wrapper_cache.cache = dict()
    return wrapper_cache


@cache  # Use it here.
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [10]:
fibonacci(8)  # Much faster and fewer calls!

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'


21

In the standard library, a Least Recently Used (LRU) cache is available as `@functools.lru_cache`.

`maxsize` parameter specifies how many recent calls are cached.
The default value is 128, but you can specify `maxsize=None` to cache all function calls.

In [11]:
import functools


@functools.lru_cache(maxsize=4)  # Use standard library implementation.
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)


fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

In [12]:
fibonacci.cache_info()

CacheInfo(hits=8, misses=11, maxsize=4, currsize=4)

#### Adding Information About Units

The following example is somewhat similar to the *Registering Plugins* example from earlier,
in that it does not really change the behavior of the decorated function.

Instead, it simply adds unit as a function attribute:

In [3]:
def set_unit(unit):
    """Register a unit on a function"""

    def decorator_set_unit(func):
        func.unit = unit
        return func

    return decorator_set_unit

In [4]:
import math


@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

In [5]:
volume(3, 5)

141.3716694115407

In [6]:
volume.unit

'cm^3'

Units become even more powerful and fun when connected with a library that can convert between units.

One such library is `pint`. With `pint` installed (`pip install Pint`), you can for instance convert the volume to cubic inches or gallons:

In [7]:
import pint

ureg = pint.UnitRegistry()
vol = volume(3, 5) * ureg(volume.unit)  # Multiplication here is the design of `pint`.

In [8]:
vol

141.3716694115407 <Unit('centimeter ** 3')>

In [9]:
vol.to("cubic inches")

8.627028576414954 <Unit('inch ** 3')>

In [10]:
vol.to("gallons").m  # Magnitude

0.0373464440537444

In [13]:
# Create a decorator to return the `pint` Quantity (value * unit from `UnitRegistry` straight away).

import functools


def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()

    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)

        return wrapper_use_unit

    return decorator_use_unit


@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [14]:
bolt = average_speed(100, 9.58)
bolt

10.438413361169102 <Unit('meter / second')>

In [16]:
bolt.to("km per hour")  # type: ignore

37.578288100208766 <Unit('kilometer / hour')>

In [18]:
bolt.to("mph").m  # type: ignore  # Magnitude

23.35006567906474

#### Validating JSON

Example for request response validation - expected arguments validation.

In [19]:
from flask import Flask, request, abort
import functools

app = Flask(__name__)


def validate_json(*expected_args):  # 1
    def decorator_validate_json(func):
        @functools.wraps(func)
        def wrapper_validate_json(*args, **kwargs):
            json_object = request.get_json()
            for expected_arg in expected_args:  # 2
                if expected_arg not in json_object:
                    abort(400)
            return func(*args, **kwargs)

        return wrapper_validate_json

    return decorator_validate_json

Above:
1. The list of keys that must be present in the JSON is given as arguments to the decorator.
2. The wrapper function validates that each expected key is present in the JSON data.

In [20]:
# Use like so:


@app.route("/grade", methods=["POST"])
@validate_json("student_id")  # <-- If "student_id" not in the response, it is not valid.
def update_grade():
    json_data = request.get_json()
    # Update database.
    return "success!"